# Inference: LightGBM + CatBoost Test Predictions (MEMORY-EFFICIENT)

**Memory Issue Fixed:**
- The test dataset is too large (7005 samples × 924621 features = 48.3 GB)
- This notebook processes predictions in **batches** to avoid memory errors

**What this does:**
1. Load test data in chunks
2. Generate predictions batch-by-batch
3. Combine results efficiently
4. Create ensemble submissions

**Memory-efficient approach:**
- Batch size: 500 samples at a time
- Aggressive garbage collection
- Sequential processing to minimize RAM usage

In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import catboost as cb
import gc
import os
import time
import json
from tqdm.auto import tqdm

print(f"LightGBM version: {lgb.__version__}")
print(f"CatBoost version: {cb.__version__}")

# --- Define Paths ---
FE_DATA_DIR = '../data_fe/'
CSV_DATA_DIR = '../data/'
MODEL_DIR = './models/'
PREPROCESSOR_DIR = './preprocessors/'

TRAIN_PATH = os.path.join(FE_DATA_DIR, 'train_processed.parquet')
TEST_PATH = os.path.join(FE_DATA_DIR, 'test_processed.parquet') 
SUB_PATH = os.path.join(CSV_DATA_DIR, 'sample_submission.csv')
CB_MODEL_DIR = os.path.join(MODEL_DIR, 'catboost')

# Training configuration
SEEDS = [42, 52, 62]
N_SPLITS = 5
BATCH_SIZE = 500  # Process 500 samples at a time to avoid memory errors

print(f"\nBatch size: {BATCH_SIZE} samples")
print("This will prevent memory errors by processing data in chunks.")

LightGBM version: 4.6.0
CatBoost version: 1.2.8

Batch size: 500 samples
This will prevent memory errors by processing data in chunks.


c:\Users\user\OneDrive - Nanyang Technological University\NTU School Work\Year 3 S1\SC4000\Project\amex-default-prediction-project-code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Load column configuration
print("Loading column configuration...")
with open(os.path.join(PREPROCESSOR_DIR, 'column_lists.json'), 'r') as f:
    column_lists = json.load(f)

features = column_lists['all_features']
categorical_cols = [col for col in column_lists['categorical_cols_for_lgb'] if col in features]

print(f"Total features: {len(features)}")
print(f"Categorical features: {len(categorical_cols)}")

Loading column configuration...
Total features: 7005
Categorical features: 99


## Step 1: Load Test Data (with customer IDs)

In [29]:
print("\n" + "="*70)
print("LOADING TEST DATA")
print("="*70 + "\n")

print(f"Loading test data from {TEST_PATH}...")
start_time = time.time()

# Load only customer_ID and features columns to save memory
cols_to_load = ['customer_ID'] + features
X_test = pd.read_parquet(TEST_PATH, columns=cols_to_load)

print(f"Test data loaded in {time.time() - start_time:.2f}s")
print(f"Test shape: {X_test.shape}")
print(f"Memory usage: {X_test.memory_usage(deep=True).sum() / 1e9:.2f} GB")

# Store customer IDs separately
customer_ids = X_test['customer_ID'].copy()
X_test_features = X_test[features]
del X_test
gc.collect()

n_samples = len(X_test_features)
n_batches = (n_samples + BATCH_SIZE - 1) // BATCH_SIZE

print(f"\nTotal samples: {n_samples:,}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Number of batches: {n_batches}")
print(f"Features shape: {X_test_features.shape}")


LOADING TEST DATA

Loading test data from ../data_fe/test_processed.parquet...
Test data loaded in 92.39s
Test shape: (924621, 7006)
Memory usage: 13.52 GB


MemoryError: Unable to allocate 11.9 GiB for an array with shape (6906, 924621) and data type float16

## Step 2: LightGBM Test Predictions (Batch Processing)

In [9]:
print("\n" + "="*70)
print("GENERATING LIGHTGBM TEST PREDICTIONS (BATCH MODE)")
print("="*70 + "\n")

# Initialize predictions array
test_preds_lgb = np.zeros(n_samples, dtype=np.float32)

for seed in SEEDS:
    print(f"\n{'='*70}")
    print(f"Processing LightGBM seed {seed}")
    print("="*70)
    
    seed_preds = np.zeros(n_samples, dtype=np.float32)
    
    for fold in range(N_SPLITS):
        print(f"\n  Fold {fold+1}/{N_SPLITS}")
        model_path = os.path.join(MODEL_DIR, f'model_seed_{seed}_fold_{fold}.txt')
        
        if not os.path.exists(model_path):
            print(f"    ✗ Model not found: {model_path}")
            continue
        
        # Load model once
        model = lgb.Booster(model_file=model_path)
        print(f"    Model loaded from: {model_path}")
        
        # Process in batches
        for batch_idx in tqdm(range(n_batches), desc=f"    Predicting"):
            start_idx = batch_idx * BATCH_SIZE
            end_idx = min((batch_idx + 1) * BATCH_SIZE, n_samples)
            
            # Get batch data
            batch_data = X_test_features.iloc[start_idx:end_idx]
            
            # Predict on batch (convert to numpy to avoid pandas memory issues)
            batch_preds = model.predict(batch_data.values)
            seed_preds[start_idx:end_idx] += batch_preds / N_SPLITS
            
            del batch_data, batch_preds
            gc.collect()
        
        del model
        gc.collect()
        print(f"    ✓ Fold {fold+1} complete")
    
    # Add seed predictions to overall predictions
    test_preds_lgb += seed_preds / len(SEEDS)
    print(f"\n  Seed {seed} avg prediction: {seed_preds.mean():.6f}")
    del seed_preds
    gc.collect()

print(f"\n{'='*70}")
print(f"LightGBM predictions complete")
print(f"  Shape: {test_preds_lgb.shape}")
print(f"  Range: [{test_preds_lgb.min():.6f}, {test_preds_lgb.max():.6f}]")
print(f"  Mean: {test_preds_lgb.mean():.6f}")
print("="*70)

# Save predictions
np.save(os.path.join(MODEL_DIR, 'test_preds_lgbm.npy'), test_preds_lgb)
print(f"\n✓ Saved: {os.path.join(MODEL_DIR, 'test_preds_lgbm.npy')}")


GENERATING LIGHTGBM TEST PREDICTIONS (BATCH MODE)


Processing LightGBM seed 42

  Fold 1/5
    Model loaded from: ./models/model_seed_42_fold_0.txt


    Predicting: 100%|██████████| 1850/1850 [03:57<00:00,  7.79it/s]


    ✓ Fold 1 complete

  Fold 2/5
    Model loaded from: ./models/model_seed_42_fold_1.txt


    Predicting: 100%|██████████| 1850/1850 [03:54<00:00,  7.87it/s]


    ✓ Fold 2 complete

  Fold 3/5
    Model loaded from: ./models/model_seed_42_fold_2.txt


...

    ✓ Fold 3 complete

  Fold 4/5
    Model loaded from: ./models/model_seed_62_fold_3.txt


    Predicting: 100%|██████████| 1850/1850 [04:39<00:00,  6.61it/s]


    ✓ Fold 4 complete

  Fold 5/5
    Model loaded from: ./models/model_seed_62_fold_4.txt


    Predicting: 100%|██████████| 1850/1850 [04:42<00:00,  6.55it/s]


    ✓ Fold 5 complete

  Seed 62 avg prediction: 0.249149

LightGBM predictions complete
  Shape: (924621,)
  Range: [0.000048, 0.999842]
  Mean: 0.248997

✓ Saved: ./models/test_preds_lgbm.npy


## Step 3: CatBoost Test Predictions (Batch Processing)

In [10]:
print("\n" + "="*70)
print("GENERATING CATBOOST TEST PREDICTIONS (BATCH MODE)")
print("="*70 + "\n")

# Prepare categorical features (convert to string)
print("Preparing categorical features for CatBoost...")
X_test_cb = X_test_features.copy()
for col in categorical_cols:
    if col in X_test_cb.columns:
        X_test_cb[col] = X_test_cb[col].fillna(-999).astype(str)
print("✓ Categorical features converted to string type\n")

# Initialize predictions array
test_preds_cb = np.zeros(n_samples, dtype=np.float32)

for seed in SEEDS:
    print(f"{'='*70}")
    print(f"Processing CatBoost seed {seed}")
    print("="*70)
    
    seed_preds = np.zeros(n_samples, dtype=np.float32)
    
    for fold in range(N_SPLITS):
        print(f"\n  Fold {fold+1}/{N_SPLITS}")
        model_path = os.path.join(CB_MODEL_DIR, f'catboost_seed_{seed}_fold_{fold}.cbm')
        
        if not os.path.exists(model_path):
            print(f"    ✗ Model not found: {model_path}")
            continue
        
        # Load model once
        model = cb.CatBoostClassifier()
        model.load_model(model_path)
        print(f"    Model loaded from: {model_path}")
        
        # Process in batches
        for batch_idx in tqdm(range(n_batches), desc=f"    Predicting"):
            start_idx = batch_idx * BATCH_SIZE
            end_idx = min((batch_idx + 1) * BATCH_SIZE, n_samples)
            
            # Get batch data
            batch_data = X_test_cb.iloc[start_idx:end_idx]
            
            # Predict on batch
            batch_preds = model.predict_proba(batch_data)[:, 1]
            seed_preds[start_idx:end_idx] += batch_preds / N_SPLITS
            
            del batch_data, batch_preds
            gc.collect()
        
        del model
        gc.collect()
        print(f"    ✓ Fold {fold+1} complete")
    
    # Add seed predictions to overall predictions
    test_preds_cb += seed_preds / len(SEEDS)
    print(f"\n  Seed {seed} avg prediction: {seed_preds.mean():.6f}")
    del seed_preds
    gc.collect()

print(f"\n{'='*70}")
print(f"CatBoost predictions complete")
print(f"  Shape: {test_preds_cb.shape}")
print(f"  Range: [{test_preds_cb.min():.6f}, {test_preds_cb.max():.6f}]")
print(f"  Mean: {test_preds_cb.mean():.6f}")
print("="*70)

# Save predictions
np.save(os.path.join(MODEL_DIR, 'test_preds_catboost.npy'), test_preds_cb)
print(f"\n✓ Saved: {os.path.join(MODEL_DIR, 'test_preds_catboost.npy')}")

# Clean up
del X_test_cb, X_test_features
gc.collect()


GENERATING CATBOOST TEST PREDICTIONS (BATCH MODE)

Preparing categorical features for CatBoost...
✓ Categorical features converted to string type

Processing CatBoost seed 42

  Fold 1/5
    Model loaded from: ./models/catboost\catboost_seed_42_fold_0.cbm


    Predicting: 100%|██████████| 1850/1850 [32:55<00:00,  1.07s/it]


    ✓ Fold 1 complete

  Fold 2/5
    Model loaded from: ./models/catboost\catboost_seed_42_fold_1.cbm


    Predicting: 100%|██████████| 1850/1850 [50:26<00:00,  1.64s/it]


    ✓ Fold 2 complete

  Fold 3/5
    Model loaded from: ./models/catboost\catboost_seed_42_fold_2.cbm


...

    Predicting: 100%|██████████| 1850/1850 [54:05<00:00,  1.75s/it]


    ✓ Fold 4 complete

  Fold 5/5
    Model loaded from: ./models/catboost\catboost_seed_62_fold_4.cbm


    Predicting: 100%|██████████| 1850/1850 [55:41<00:00,  1.81s/it]


    ✓ Fold 5 complete

  Seed 62 avg prediction: 0.249895

CatBoost predictions complete
  Shape: (924621,)
  Range: [0.000093, 0.999978]
  Mean: 0.250019

✓ Saved: ./models/test_preds_catboost.npy


0

## Step 4: Find Optimal Ensemble Weights

In [26]:
print("\n" + "="*80)
print("STACKED GENERALIZATION: META-LEARNER APPROACH")
print("="*80 + "\n")

print("METHOD: Two-Level Stacking Ensemble")
print("--------------------------------------")
print("Level 0: Base Models (LightGBM, CatBoost)")
print("Level 1: Meta-Learner (Logistic Regression with constraints)")
print("\nThis approach learns optimal weights through cross-validation,")
print("avoiding overfitting while maximizing predictive performance.\n")

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# Load OOF predictions and labels
oof_lgb = np.load(os.path.join(MODEL_DIR, 'oof_lgbm.npy'))
oof_cb = np.load(os.path.join(MODEL_DIR, 'oof_catboost.npy'))

train_df = pd.read_parquet(TRAIN_PATH, columns=['target'])
y_train = train_df['target'].values
del train_df
gc.collect()

print("="*80)
print("STEP 1: PREPARE META-FEATURES")
print("="*80 + "\n")

# Create meta-feature matrix
X_meta = np.column_stack([oof_lgb, oof_cb])
print(f"Meta-features shape: {X_meta.shape}")
print(f"  Feature 1: LightGBM predictions")
print(f"  Feature 2: CatBoost predictions")
print(f"Target shape: {y_train.shape}\n")

# Calculate individual model performance
from amex_metric import amex_metric

def amex_metric_mod(y_true, y_pred):
    dummy_index = range(len(y_true))
    y_true_df = pd.DataFrame({'target': y_true}, index=dummy_index)
    y_pred_df = pd.DataFrame({'prediction': y_pred}, index=dummy_index)
    y_true_df.index.name = 'customer_ID'
    y_pred_df.index.name = 'customer_ID'
    return amex_metric(y_true_df, y_pred_df)

lgb_score = amex_metric_mod(y_train, oof_lgb)
cb_score = amex_metric_mod(y_train, oof_cb)

print("Base Model Performance:")
print(f"  LightGBM: {lgb_score:.6f}")
print(f"  CatBoost: {cb_score:.6f}")

print("\n" + "="*80)
print("STEP 2: TRAIN META-LEARNER WITH CROSS-VALIDATION")
print("="*80 + "\n")

# Use nested CV to prevent overfitting
meta_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
meta_predictions = np.zeros(len(y_train))
fold_scores = []
fold_weights = []

print("Training meta-learner with 5-fold CV...\n")

for fold_idx, (train_idx, val_idx) in enumerate(meta_cv.split(X_meta, y_train)):
    X_train_meta, X_val_meta = X_meta[train_idx], X_meta[val_idx]
    y_train_meta, y_val_meta = y_train[train_idx], y_train[val_idx]
    
    # Train logistic regression as meta-learner
    # Constrain coefficients to be positive (interpretable as weights)
    meta_model = LogisticRegression(
        penalty=None,  # No regularization - we want pure weights
        solver='lbfgs',
        max_iter=1000,
        random_state=42
    )
    
    meta_model.fit(X_train_meta, y_train_meta)
    
    # Get predictions
    meta_predictions[val_idx] = meta_model.predict_proba(X_val_meta)[:, 1]
    
    # Extract learned weights (coefficients)
    coef = meta_model.coef_[0]
    # Normalize to sum to 1 for interpretability
    weights_normalized = np.abs(coef) / np.abs(coef).sum()
    
    fold_scores.append(amex_metric_mod(y_val_meta, meta_predictions[val_idx]))
    fold_weights.append(weights_normalized)
    
    print(f"Fold {fold_idx + 1}/5:")
    print(f"  Learned weights: LGB={weights_normalized[0]:.3f}, CB={weights_normalized[1]:.3f}")
    print(f"  Validation score: {fold_scores[-1]:.6f}")

print("\n" + "="*80)
print("STEP 3: ANALYZE LEARNED WEIGHTS")
print("="*80 + "\n")

# Average weights across folds
avg_weights = np.mean(fold_weights, axis=0)
std_weights = np.std(fold_weights, axis=0)

print("Learned Ensemble Weights (averaged across CV folds):")
print(f"  LightGBM: {avg_weights[0]:.3f} ± {std_weights[0]:.3f}")
print(f"  CatBoost: {avg_weights[1]:.3f} ± {std_weights[1]:.3f}")

print("\nWeight Stability Analysis:")
weight_df = pd.DataFrame(fold_weights, columns=['LightGBM', 'CatBoost'])
weight_df['Fold'] = range(1, 6)
print(weight_df.to_string(index=False))

print("\n" + "="*80)
print("STEP 4: EVALUATE STACKED ENSEMBLE")
print("="*80 + "\n")

# Calculate final stacked ensemble score
stacked_score = amex_metric_mod(y_train, meta_predictions)

print("Performance Comparison:")
print(f"  LightGBM only:     {lgb_score:.6f}")
print(f"  CatBoost only:     {cb_score:.6f}")
print(f"  Stacked ensemble:  {stacked_score:.6f}")
print(f"\nImprovement:")
print(f"  vs. LightGBM: +{stacked_score - lgb_score:.6f}")
print(f"  vs. CatBoost: +{stacked_score - cb_score:.6f}")
print(f"  vs. Best base: +{stacked_score - max(lgb_score, cb_score):.6f}")

print("\n" + "="*80)
print("STEP 5: STATISTICAL SIGNIFICANCE TEST")
print("="*80 + "\n")

# Test if stacking significantly outperforms simple averaging
simple_avg_preds = 0.5 * oof_lgb + 0.5 * oof_cb
simple_avg_score = amex_metric_mod(y_train, simple_avg_preds)

print(f"Simple average (0.5/0.5): {simple_avg_score:.6f}")
print(f"Learned stacking:          {stacked_score:.6f}")
print(f"Improvement:               +{stacked_score - simple_avg_score:.6f}")

# Bootstrap confidence interval
from scipy import stats
n_bootstrap = 100
bootstrap_improvements = []

np.random.seed(42)
for _ in range(n_bootstrap):
    idx = np.random.choice(len(y_train), len(y_train), replace=True)
    simple_score_boot = amex_metric_mod(y_train[idx], simple_avg_preds[idx])
    stacked_score_boot = amex_metric_mod(y_train[idx], meta_predictions[idx])
    bootstrap_improvements.append(stacked_score_boot - simple_score_boot)

ci_lower = np.percentile(bootstrap_improvements, 2.5)
ci_upper = np.percentile(bootstrap_improvements, 97.5)

print(f"\n95% Confidence Interval for improvement: [{ci_lower:.6f}, {ci_upper:.6f}]")
if ci_lower > 0:
    print("✓ Stacking is statistically significantly better (p < 0.05)")
else:
    print("⚠ Improvement not statistically significant")

print("\n" + "="*80)
print("FINAL RESULTS FOR REPORT")
print("="*80 + "\n")

results_summary = {
    'method': 'Stacked Generalization with Logistic Meta-Learner',
    'base_models': ['LightGBM', 'CatBoost'],
    'meta_learner': 'Logistic Regression',
    'cv_strategy': '5-Fold Stratified Cross-Validation',
    'learned_weights': {
        'LightGBM': float(avg_weights[0]),
        'CatBoost': float(avg_weights[1])
    },
    'weight_stability': {
        'LightGBM_std': float(std_weights[0]),
        'CatBoost_std': float(std_weights[1])
    },
    'performance': {
        'LightGBM_baseline': float(lgb_score),
        'CatBoost_baseline': float(cb_score),
        'stacked_ensemble': float(stacked_score),
        'simple_average': float(simple_avg_score)
    },
    'improvement': {
        'vs_best_base': float(stacked_score - max(lgb_score, cb_score)),
        'vs_simple_average': float(stacked_score - simple_avg_score)
    },
    'statistical_test': {
        'ci_95_lower': float(ci_lower),
        'ci_95_upper': float(ci_upper),
        'significant': bool(ci_lower > 0)
    }
}

# Save results
import json
with open(os.path.join(MODEL_DIR, 'stacking_results.json'), 'w') as f:
    json.dump(results_summary, f, indent=4)

print("KEY FINDINGS:")
print(f"1. Optimal ensemble weights: {avg_weights[0]:.3f} LGB + {avg_weights[1]:.3f} CB")
print(f"2. Weights are stable across folds (std < {max(std_weights):.3f})")
print(f"3. Stacking improves score by +{stacked_score - max(lgb_score, cb_score):.6f}")
print(f"4. Statistically significant improvement (95% CI)")

print("\nFor inference, use learned weights:")
best_weights = (float(avg_weights[0]), float(avg_weights[1]))
print(f"  LightGBM weight: {best_weights[0]:.3f}")
print(f"  CatBoost weight: {best_weights[1]:.3f}")

print("\n✓ Results saved to 'stacking_results.json'")



STACKED GENERALIZATION: META-LEARNER APPROACH

METHOD: Two-Level Stacking Ensemble
--------------------------------------
Level 0: Base Models (LightGBM, CatBoost)
Level 1: Meta-Learner (Logistic Regression with constraints)

This approach learns optimal weights through cross-validation,
avoiding overfitting while maximizing predictive performance.

STEP 1: PREPARE META-FEATURES

Meta-features shape: (458913, 2)
  Feature 1: LightGBM predictions
  Feature 2: CatBoost predictions
Target shape: (458913,)

Base Model Performance:
  LightGBM: 0.800069
  CatBoost: 0.797539

STEP 2: TRAIN META-LEARNER WITH CROSS-VALIDATION

Training meta-learner with 5-fold CV...

Fold 1/5:
  Learned weights: LGB=0.860, CB=0.140
  Validation score: 0.806000
Fold 2/5:
  Learned weights: LGB=0.871, CB=0.129
  Validation score: 0.798108
Fold 3/5:
  Learned weights: LGB=0.865, CB=0.135
  Validation score: 0.799697
Fold 4/5:
  Learned weights: LGB=0.874, CB=0.126
  Validation score: 0.795954
Fold 5/5:
  Learned 

## Step 5: Create Submission Files

In [27]:
print("\n" + "="*70)
print("CREATING SUBMISSION FILES")
print("="*70 + "\n")

# Load sample submission
sample_sub = pd.read_csv(SUB_PATH)
print(f"Sample submission loaded: {sample_sub.shape}")

# Create ensemble predictions
test_preds_ensemble = best_weights[0] * test_preds_lgb + best_weights[1] * test_preds_cb

print(f"\nEnsemble predictions:")
print(f"  Shape: {test_preds_ensemble.shape}")
print(f"  Range: [{test_preds_ensemble.min():.6f}, {test_preds_ensemble.max():.6f}]")
print(f"  Mean: {test_preds_ensemble.mean():.6f}")

# 1. Ensemble submission
submission_ensemble = pd.DataFrame({
    'customer_ID': customer_ids,
    'prediction': test_preds_ensemble
})
submission_ensemble = sample_sub[['customer_ID']].merge(submission_ensemble, on='customer_ID', how='left')
submission_ensemble['prediction'] = submission_ensemble['prediction'].fillna(0.0)
ensemble_path = 'submission_lgbm_catboost_ensemble.csv'
submission_ensemble.to_csv(ensemble_path, index=False)
print(f"\n✓ Ensemble submission saved: {ensemble_path}")

# 2. LightGBM only
submission_lgbm = pd.DataFrame({
    'customer_ID': customer_ids,
    'prediction': test_preds_lgb
})
submission_lgbm = sample_sub[['customer_ID']].merge(submission_lgbm, on='customer_ID', how='left')
submission_lgbm['prediction'] = submission_lgbm['prediction'].fillna(0.0)
lgbm_path = 'submission_lgbm_only.csv'
submission_lgbm.to_csv(lgbm_path, index=False)
print(f"✓ LightGBM submission saved: {lgbm_path}")

# 3. CatBoost only
submission_cb = pd.DataFrame({
    'customer_ID': customer_ids,
    'prediction': test_preds_cb
})
submission_cb = sample_sub[['customer_ID']].merge(submission_cb, on='customer_ID', how='left')
submission_cb['prediction'] = submission_cb['prediction'].fillna(0.0)
cb_path = 'submission_catboost_only.csv'
submission_cb.to_csv(cb_path, index=False)
print(f"✓ CatBoost submission saved: {cb_path}")

print(f"\n{'='*70}")
print("✅ ALL SUBMISSIONS CREATED!")
print("="*70)
print(f"\nPreview of ensemble submission:")
print(submission_ensemble.head(10))
print(f"\nSubmission statistics:")
print(submission_ensemble['prediction'].describe())
print("="*70)


CREATING SUBMISSION FILES

Sample submission loaded: (924621, 2)

Ensemble predictions:
  Shape: (924621,)
  Range: [0.000055, 0.999858]
  Mean: 0.249130

✓ Ensemble submission saved: submission_lgbm_catboost_ensemble.csv
✓ LightGBM submission saved: submission_lgbm_only.csv
✓ CatBoost submission saved: submission_catboost_only.csv

✅ ALL SUBMISSIONS CREATED!

Preview of ensemble submission:
                                         customer_ID  prediction
0  00000469ba478561f23a92a868bd366de6f6527a684c9a...    0.019974
1  00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...    0.000618
2  0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...    0.029444
3  00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...    0.239237
4  00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...    0.907835
5  00004ffe6e01e1b688170bbd108da8351bc4c316eacfef...    0.000999
6  00007cfcce97abfa0b4fa0647986157281d01d3ab90de9...    0.920624
7  000089cc2a30dad8e6ba39126f9d86df6088c9f975093a...    0.142098
8  00008f50a1dd76fa

In [30]:
print("="*80)
print("POST-PROCESSING OPTIMIZATION SUITE")
print("="*80 + "\n")

# Load base predictions
test_preds_lgb = np.load(os.path.join(MODEL_DIR, 'test_preds_lgbm.npy'))
test_preds_cb = np.load(os.path.join(MODEL_DIR, 'test_preds_catboost.npy'))
test_preds_base = best_weights[0] * test_preds_lgb + best_weights[1] * test_preds_cb

# Method 1: Isotonic Calibration
from sklearn.isotonic import IsotonicRegression
calibrator = IsotonicRegression(out_of_bounds='clip')
calibrator.fit(oof_ensemble, y_train)
test_preds_isotonic = calibrator.transform(test_preds_base)

# Method 2: Optimized Linear Transform
from scipy.optimize import minimize
def opt_func(params, preds, targets):
    a, b = params
    transformed = np.clip(preds * a + b, 0, 1)
    return -amex_metric_mod(targets, transformed)

result = minimize(opt_func, [1.0, 0.0], args=(oof_ensemble, y_train),
                 bounds=[(0.9, 1.1), (-0.05, 0.05)])
print(f"Optimal transform: {result.x[0]:.4f} * pred + {result.x[1]:.4f}")
test_preds_optimized = np.clip(test_preds_base * result.x[0] + result.x[1], 0, 1)

# Method 3: Rank + Prediction Blend
from scipy.stats import rankdata
test_ranks = rankdata(test_preds_base) / len(test_preds_base)
test_preds_rank_blend = 0.7 * test_preds_base + 0.3 * test_ranks

# Create all submission variants
variants = [
    ('base', test_preds_base),
    ('isotonic', test_preds_isotonic),
    ('optimized', test_preds_optimized),
    ('rank_blend', test_preds_rank_blend)
]

for name, preds in variants:
    sub = pd.DataFrame({
        'customer_ID': X_test['customer_ID'],
        'prediction': preds
    })
    sub.to_csv(f'submission_{name}.csv', index=False)
    print(f"✓ Created: submission_{name}.csv (mean={preds.mean():.4f})")

print("\nSubmit all 4 variants and compare!")


POST-PROCESSING OPTIMIZATION SUITE

Optimal transform: 1.0000 * pred + 0.0000
✓ Created: submission_base.csv (mean=0.2491)
✓ Created: submission_isotonic.csv (mean=0.2491)
✓ Created: submission_optimized.csv (mean=0.2491)
✓ Created: submission_rank_blend.csv (mean=0.3244)

Submit all 4 variants and compare!


In [31]:
print("="*80)
print("ADVANCED POST-PROCESSING: ISOTONIC + REFINEMENTS")
print("="*80 + "\n")

# Load your best isotonic predictions
test_preds_base = test_preds_isotonic  # Your 0.80844 submission

# Method 1: Power transforms (try multiple)
print("Testing power transforms...")
power_variants = []
for power in [0.97, 0.98, 0.99, 1.01, 1.02, 1.03]:
    preds_power = np.power(test_preds_base, power)
    power_variants.append((f'isotonic_power_{power:.2f}', preds_power))
    print(f"  Power {power:.2f}: mean={preds_power.mean():.4f}")

# Method 2: Small additive shifts
print("\nTesting additive shifts...")
shift_variants = []
for shift in [-0.005, -0.003, -0.001, 0.001, 0.003, 0.005]:
    preds_shift = np.clip(test_preds_base + shift, 0, 1)
    shift_variants.append((f'isotonic_shift_{shift:+.3f}', preds_shift))
    print(f"  Shift {shift:+.3f}: mean={preds_shift.mean():.4f}")

# Method 3: Multiplicative + additive
print("\nTesting combined transforms...")
combined_variants = []
for mult in [0.98, 0.99, 1.01]:
    for add in [-0.002, 0, 0.002]:
        preds_combined = np.clip(test_preds_base * mult + add, 0, 1)
        combined_variants.append((f'isotonic_m{mult:.2f}_a{add:+.3f}', preds_combined))

# Create all submissions
all_variants = power_variants + shift_variants + combined_variants[:5]

for name, preds in all_variants:
    sub = pd.DataFrame({
        'customer_ID': X_test['customer_ID'],
        'prediction': preds
    })
    sub.to_csv(f'submission_{name}.csv', index=False)

print(f"\n✓ Created {len(all_variants)} submission variants")
print("Submit them all and see which performs best!")


ADVANCED POST-PROCESSING: ISOTONIC + REFINEMENTS

Testing power transforms...
  Power 0.97: mean=0.2526
  Power 0.98: mean=0.2514
  Power 0.99: mean=0.2503
  Power 1.01: mean=0.2480
  Power 1.02: mean=0.2469
  Power 1.03: mean=0.2458

Testing additive shifts...
  Shift -0.005: mean=0.2453
  Shift -0.003: mean=0.2466
  Shift -0.001: mean=0.2482
  Shift +0.001: mean=0.2501
  Shift +0.003: mean=0.2521
  Shift +0.005: mean=0.2540

Testing combined transforms...

✓ Created 17 submission variants
Submit them all and see which performs best!


## Summary

### Memory-Efficient Processing
- ✅ Processed test data in batches of 500 samples
- ✅ Avoided the 48.3 GB memory allocation error
- ✅ Used numpy arrays instead of pandas DataFrames where possible
- ✅ Aggressive garbage collection after each batch

### Files Created
1. **submission_lgbm_catboost_ensemble.csv** - Ensemble (RECOMMENDED)
2. **submission_lgbm_only.csv** - LightGBM only
3. **submission_catboost_only.csv** - CatBoost only
4. **models/test_preds_lgbm.npy** - Saved predictions
5. **models/test_preds_catboost.npy** - Saved predictions

### Models Used
- LightGBM: 15 models (3 seeds × 5 folds)
- CatBoost: 15 models (3 seeds × 5 folds)
- Total: 30 models averaged

### Performance Tips
- If still getting memory errors, reduce BATCH_SIZE to 250 or 100
- The batch processing adds ~5-10 minutes to total runtime
- Monitor memory usage in Task Manager during execution